In [1]:
import brownie

Attached to local RPC client listening at '127.0.0.1:8545'...


In [26]:
brownie.network.connect("development")
myproject = brownie.project.load("/data/tutorial_brownie/hello_world_contract")

pragma solidity ^0.5.11;

contract SimpleContract {
     uint value;
     function setValue(uint _value) external {value = _value;}
     function getValue() external view returns(uint){return value;}       
}


In [25]:
print("SimpleContract=", myproject.SimpleContract)